In [ ]:
!pip install finance-datareader
!pip install opendartreader
!pip install --upgrade opendartreader

=========== 오류 코드 수정 9.15 ========   2017~2021 실행 가능 확인[ 업종 코드 = 5380 5930 51910,660,68270] 

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr

# 재무재표 로딩
import OpenDartReader

import datetime

In [2]:
def pro_equ(code, start, end):
    # OpenDartReader 사용
    api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
    dart = OpenDartReader(api_key)
    arr = [] # 재무 정보
    index = [] # 계산지표
    
    # 주식정보
    if type(code) != str:
        code = str(code).zfill(6)       
    
    if type(start) != str:
        start = str(start)
    
    if type(end) != str:
        end = str(end)
        
    stock = fdr.DataReader(code, start = str(int(start)+1), end = end)  # 2021 지표 <- 2020,2019 데이터로 만듬
    stock = stock.reset_index()
    stock['year'] = stock['Date'].dt.year
    stock['PER'] = 0.1
    stock['PBR'] = 0.1
    stock['ROE'] = 0.1
    stock['ROA'] = 0.1
#     print(stock.info())
    #         ------------------------
    
    small = dart.report('005930', '소액주주', int(datetime.datetime.now().strftime("%Y"))-1, reprt_code=11014)
    stock_tot_co = int(small['stock_tot_co'].str.replace(',', '')) # 총 발행 주식 수
    
    for i in range(int(start[:4]), int(end[:4])):
        # https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019016 --> 입력변수 명 참고
        # corp: 종목코드, bsns_year: 사업연도, reprt_code: 보고서코드
        # (1분기보고서 : 11013, 반기보고서 : 11012, 3분기보고서 : 11014, 사업보고서 : 11011)
        fs_before = dart.finstate_all(corp = code, bsns_year= str(i-1), fs_div='CFS', reprt_code=11011) 
        fs_present = dart.finstate_all(corp = code, bsns_year= i, fs_div='CFS', reprt_code=11014)

        # 자본, 부채는 당기 금액('thstrm_amount') 값을 통해서 구함
#         equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        equity = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['자본총계']), 'thstrm_amount'])
#         print(equity)
#         liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        liability = int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['부채총계']), 'thstrm_amount'].replace(",", ""))
#         print(liability)
        # 자산 총계
        assets = equity + liability
        
        # <예시>
        # 2019 4분기 ~ 2020 3분기까지의 당기순이익의 합을 구하려면 2019년 4분기 당기순이익과 2020년 1분기 ~ 3분기 당기순이익의 합을 알아여함
        # 2020년 1분기 ~ 3분기 당기순이익의 합은 2020년 3분기 손익계산서에서 'thstrm_add_amount' 값을 가져오면 되고
        # 2019년 4분기 당기순이익은 2019년 전체 당기순이익에서 2019년 1분기 ~ 3분기 당기순이익의 합을 빼서 구할 수 있음
        
        # frmtrm_add_amount --> 전기 금액, thstrm_amount ---> 당기 금액, thstrm_add_amount --> 당기 누적금액
#         profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "")) # 당기순이익
        
    
        sj_l = ['IS','CIS']
        sj = []
        sj_b =[]
        for re in range(len(sj_l)):
            if sj_l[re] in list(fs_present['sj_div'].unique()):
                sj.append(sj_l[re])
            if sj_l[re] in list(fs_before['sj_div'].unique()):
                sj_b.append(sj_l[re])
        
        
        pf_l = ['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익','분(반)기순이익(손실)','순이익(손실)','분기순이익(손실)','연결분기순이익','연결당기순이익', '연결분기순이익(손실)']
        for k in range(len(pf_l)):
            if pf_l[k] in list(fs_present.loc[fs_present['sj_div'].isin(sj),'account_nm'].unique()):
                pf = pf_l[k]
            if pf_l[k] in list(fs_before['account_nm'].unique()):
                pf_b = pf_l[k]     
        
        
#         print(pf)
        profit_before_3Q = int(fs_present.loc[fs_present['sj_div'].isin(sj) & fs_present['account_nm'].isin([pf]), 'frmtrm_add_amount'].iloc[0].replace(",", ""))
#         print(profit_before_3Q)
#         profit_before = int(fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")) # 당기순이익
        profit_before = int(fs_before.loc[fs_before['sj_div'].isin(sj_b) & fs_before['account_nm'].isin([pf_b]), 'thstrm_amount'].iloc[0].replace(",", ""))
#         print(profit_before)
#         profit_present = int(fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")) # 당기순이익
        profit_present = int(fs_present.loc[fs_present['sj_div'].isin(sj) & fs_present['account_nm'].isin([pf]), 'thstrm_add_amount'].iloc[0].replace(",", "")) # 당기순이익
#         print(profit_present)
        profit = (profit_before-profit_before_3Q) + profit_present
        
        a = (equity, liability, assets, profit, stock_tot_co) 
        # 자본총계(당기자본), 부채총계(당기부채), 자산총계, 당기 순이익, 총 발행 주식수
        arr.append(a)
        
#         - PER(주가 수익률) : 주가를 1주당 순이익(EPS)로 나눈 값
#         - PBR(주가 순자산 비율) : 주가를 1주당 순자산가치로 나눈 값
#         - ROE(자기자본이익률) : 당기순이익을 순자산(자기자본)으로 나눈 값
#         - ROA(총자산수익률) : 당기순이익을 자산총액으로 나눈 값
        
    
        ROA = profit/assets
        stock['ROA'][stock[stock.year == i+1].index] = ROA
        
        
        EPS = profit/stock_tot_co
        PER = stock['Close'][stock[stock.year == i+1].index]/EPS
        stock['PER'][stock[stock.year == i+1].index] = PER

#         stock['PER'][stock[stock['Date'].dt.year == i+1].index] = PER
#         PER = stock[stock['Date'].dt.year == i+1]['Close']/EPS
        
        BPS = equity/stock_tot_co
        PBR = stock['Close'][stock[stock.year == i+1].index]/BPS
#         PBR = stock[stock['Date'].dt.year == i+1]['Close']/BPS
        stock['PBR'][stock[stock.year == i+1].index] = PBR
    
    
    
#         for i in PER.index:
#             stock['PER'][i] = PER[i]
#             stock['PBR'][i] = PBR[i]
        
        b = (EPS,BPS)
        # EPS: 순이익/총발행주식수 --> 주당 수익?
        # BPS: 순자산/총발행 주식수 --> 주당 자산가치?
        
        index.append(b)
        
    stock['ROE'] = stock['PBR']/stock['PER']
    
    return arr,index, stock

In [3]:
end_date = datetime.datetime.now().strftime("%Y%m%d")

In [4]:
arr, indexs, df = pro_equ(68270,2017,end_date)  # 2018의 경우 2019,2020,2021까지의 결과 도출

pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인
pf 미확인


In [5]:
df

,Date,Open,High,Low,Close,Volume,Change,year,PER,PBR,ROE,ROA
0,2018-01-02,206047,209825,204757,208167,1176443,0.021709,2018,3583.888333,504.641965,0.140809,0.105425
1,2018-01-03,210931,227794,209272,227334,2411162,0.092075,2018,3913.875256,551.106931,0.140809,0.105425
2,2018-01-04,230007,240696,226044,230376,3265527,0.013381,2018,3966.247574,558.481399,0.140809,0.105425
3,2018-01-05,234890,246041,234061,245949,1920918,0.067598,2018,4234.358720,596.233729,0.140809,0.105425
4,2018-01-08,256637,279399,255440,278754,4298770,0.133381,2018,4799.143036,675.760166,0.140809,0.105425
...,...,...,...,...,...,...,...,...,...,...,...,...
910,2021-09-09,272000,275500,267000,269000,595228,-0.016453,2021,3119.522460,478.721765,0.153460,0.110985
911,2021-09-10,269500,271000,265500,267500,394088,-0.005576,2021,3102.127354,476.052313,0.153460,0.110985
912,2021-09-13,266500,267000,260500,262000,505510,-0.020561,2021,3038.345296,466.264321,0.153460,0.110985
913,2021-09-14,261500,265000,260500,263000,416684,0.003817,2021,3049.942034,468.043956,0.153460,0.110985


/////////////////

In [ ]:
api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
dart = OpenDartReader(api_key)

In [ ]:
stock = fdr.DataReader('068270', start = '20180101', end = end_date)

In [ ]:
fs_present3 = dart.finstate_all(corp = '005380', bsns_year= '2017', fs_div='CFS', reprt_code=11011)

In [ ]:
fs_present2 = dart.finstate_all(corp = '005380', bsns_year= '2016', fs_div='CFS', reprt_code=11011)

In [ ]:
fs_present2 = dart.finstate_all(corp = '068270', bsns_year= '2017', fs_div='CFS', reprt_code=11014)

In [ ]:
fs_present3[fs_present3['sj_div'].isin(['IS','CIS']) & fs_present3['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익','분(반)기순이익(손실)'])]

In [ ]:
fs_present2[fs_present2['sj_div'].isin(['IS','CIS']) & fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익','분(반)기순이익(손실)'])]

In [ ]:
fs_present2.loc[fs_present2['sj_div'].isin(['IS','CIS']),'account_nm'].unique()

In [ ]:
fs_present2[fs_present2['sj_div'].isin(['IS','CIS'])]

In [ ]:
fs_present2[fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익'])]

In [ ]:
fs_present2.loc[fs_present2['sj_div'].isin(['IS','CIS']) & fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익']), 'frmtrm_add_amount'].replace(",", "")

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS','CIS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익','당기순이익(손실)','분기순이익']), 'frmtrm_add_amount'].replace(",", "")

In [ ]:
fs_present = dart.finstate_all(corp = '000660', bsns_year= '2019', fs_div='CFS', reprt_code=11014)

In [ ]:
fs_present2[fs_present2['sj_div'].isin(['IS','CIS'])]

In [ ]:
16629519000000- 4487454000000 == 12142065000000

In [ ]:
fs_present[fs_present['sj_div'].isin(['IS','CIS'])]

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS','CIS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익']), 'thstrm_amount'].replace(",", "")

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS','CIS']) & fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)','당기순이익']), 'frmtrm_add_amount']

In [ ]:
fs_present2[fs_present2['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)'])]

In [ ]:
fs_present.sj_div.value_counts()

In [ ]:
fs_present[fs_present['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)'])]

In [ ]:
fs_present[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity'])]

In [ ]:
fs_present[fs_present['sj_div'].isin(['BS'])]

In [ ]:
fs_present['account_nm'].value_counts()

In [ ]:
df

In [ ]:
arr

In [ ]:
indexs

### ============= 연습 =============

In [ ]:
stock.info()

In [ ]:
stock = stock.reset_index()

In [ ]:
pd.merge(stock,PER)

In [ ]:
PER = stock[stock['Date'].dt.year == 2021]['Close']/10
for i in PER.index:
    stock['PER'][i] = PER[i]

In [ ]:
stock['per'] = 0.1

In [ ]:
a,b = 15,10

In [ ]:
stock.info()

In [ ]:
stock.loc[stock['Date'].dt.year == 2020,'per'] = b

In [ ]:
stock

In [ ]:
stock[stock['Date'].dt.year == 2020]

In [ ]:
PER[738]

In [ ]:
stock['PER'][738]

In [ ]:
stock['PER'][738] = PER[738]

In [ ]:
stock['PER'][738]

In [ ]:
PER.index

In [ ]:
stock[stock['Date'].dt.year == 2021]['PER'] = PER

In [ ]:
stock[stock['Date'].dt.year == 2021]['PER']

In [ ]:
stock

In [ ]:
a =stock.copy()
a=a.reset_index()

In [ ]:
s = a[a['Date'].dt.year == 2018]['Close']

In [ ]:
a['aaaa'] = 0.1

In [ ]:
a.head()

In [ ]:
a['aaaa'] = s

In [ ]:
a.loc[a['Date'].dt.year == 2018,'Close']/1

In [ ]:
a.loc[a['sj_div'].isin(['BS']) & a['account_nm'].isin(['부채총계']),'thstrm_amount']

In [ ]:
dart.report('005930', '소액주주', 2020, reprt_code=11014)

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['부채총계']), 'thstrm_amount'].replace(",", ""))

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", ""))

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_nm'].isin(['자본총계']), 'thstrm_amount']

=============코드 수정=======오류 파악============

In [ ]:
api_key = '9b9157d4313a84f3141d73f879c5d239d49bb0ce'
dart = OpenDartReader(api_key)

In [ ]:
fs_before = dart.finstate_all(corp = '005930', bsns_year= '2020', fs_div='CFS', reprt_code=11011) 
fs_present = dart.finstate_all(corp = '005930', bsns_year= '2020',fs_div='CFS', reprt_code=11014)

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount']

In [ ]:
fs_present.iloc[52]

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']

In [ ]:
fs_present.iloc[42]

In [ ]:
a = dart.finstate_all(corp = '005930', bsns_year= '2019')

In [ ]:
a.loc[a['sj_div'].isin(['BS']) & a['account_nm'].isin(['부채총계']),'thstrm_amount']

In [ ]:
frmtrm_add_amount

In [ ]:
fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", "")

In [ ]:
fs_before.loc[fs_before['sj_div'].isin(['IS']) & fs_before['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")

In [ ]:
fs_before.iloc[68]

In [ ]:
a = dart.finstate_all(corp = '005930', bsns_year= '2018',fs_div='CFS', reprt_code=11011)

In [ ]:
a.loc[a['sj_div'].isin(['IS']) & a['account_nm'].isin(['지배기업의 소유주에게 귀속되는 당기순이익(손실)']), 'thstrm_amount'].replace(",", "")

In [ ]:
a = dart.finstate_all(corp = '005930', bsns_year= '2020',fs_div='CFS', reprt_code=11011)

In [ ]:
fs_present.columns

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "") # 당기순이익

In [ ]:
fs_present.iloc[68]

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['IS']) & fs_present['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")

In [ ]:
fs_present['sj_div'].isin(['BS']).index

In [ ]:
fs_present['account_id'].isin(['기말자본']).index

In [ ]:
int(fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", ""))

In [ ]:
fs_present.loc[fs_present['sj_div'].isin(['BS']) & fs_present['account_id'].isin(['ifrs-full_Equity'])]

In [ ]:
fs_2019 = dart.finstate_all(corp='005930', bsns_year='2019', fs_div='CFS', reprt_code=11011) 
fs_2020_3Q = dart.finstate_all(corp='005930', bsns_year='2020', fs_div='CFS', reprt_code=11014) 



In [ ]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['IS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", ""))


In [ ]:
int(fs_2020_3Q.loc[fs_2020_3Q['sj_div'].isin(['IS']) & fs_2020_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", ""))


In [ ]:
fs_present['account_id'].isin(['ifrs-full_Equity'])